In [4]:
import feather
import scipy as sp
import numpy as np
import pandas as pd
import lightgbm as lgb

from collections import Counter
from functools import partial
from math import sqrt
from scipy.stats import rankdata

from sklearn.metrics import cohen_kappa_score, mean_squared_error
from sklearn.metrics import confusion_matrix as sk_cmatrix
from sklearn.model_selection import StratifiedKFold, GroupKFold
from sklearn.ensemble import ExtraTreesRegressor

import matplotlib.pyplot as plt
import seaborn as sns

def get_score(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

def get_y():
    return pd.read_csv('../input/petfinder-adoption-prediction/train/train.csv', usecols=[target]).values.flatten()
    
def run_extra_tree_model(X_train, y_train, X_valid, y_valid, #X_test
                        ):
    
    model = ExtraTreesRegressor(**Extra_Tree_PARAMS)
    model.fit(X_train, y_train)
    
    # train score
    y_pred_train = model.predict(X_train)
    train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))

    # validation score
    y_pred_valid = model.predict(X_valid)
    valid_rmse = np.sqrt(mean_squared_error(y_valid, y_pred_valid))
    y_pred_valid = rankdata(y_pred_valid)/len(y_pred_valid)


    # predict test
    #y_pred_test = model.predict(X_test)
    #y_pred_test = rankdata(y_pred_test)/len(y_pred_test)

    return y_pred_valid, train_rmse, valid_rmse
 

def to_bins(x, borders):
    for i in range(len(borders)):
        if x <= borders[i]:
            return i
    return len(borders)

class OptimizedRounder_(object):
    def __init__(self):
        self.coef_ = 0

    def _loss(self, coef, X, y, idx):
        X_p = np.array([to_bins(pred, coef) for pred in X])
        ll = -get_score(y, X_p)
        return ll

    def fit(self, X, y):
        coef = [1.5, 2.0, 2.5, 3.0]
        golden1 = 0.618
        golden2 = 1 - golden1
        ab_start = [(1, 2), (1.5, 2.5), (2, 3), (2.5, 3.5)]
        for it1 in range(10):
            for idx in range(4):
                # golden section search
                a, b = ab_start[idx]
                # calc losses
                coef[idx] = a
                la = self._loss(coef, X, y, idx)
                coef[idx] = b
                lb = self._loss(coef, X, y, idx)
                for it in range(20):
                    # choose value
                    if la > lb:
                        a = b - (b - a) * golden1
                        coef[idx] = a
                        la = self._loss(coef, X, y, idx)
                    else:
                        b = b - (b - a) * golden2
                        coef[idx] = b
                        lb = self._loss(coef, X, y, idx)
        self.coef_ = {'x': coef}

    def predict(self, X, coef):
        X_p = np.array([to_bins(pred, coef) for pred in X])
        return X_p

    def coefficients(self):
        return self.coef_['x']
    
class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0

    def _loss(self, coef, X, y, idx):
        X_p = np.array([to_bins(pred, coef) for pred in X])
        ll = -get_score(y, X_p)
        return ll

    def fit(self, X, y):
        coef = [0.2, 0.4, 0.6, 0.8]
        golden1 = 0.618
        golden2 = 1 - golden1
        ab_start = [(0.01, 0.3), (0.15, 0.56), (0.35, 0.75), (0.6, 0.9)]
        for it1 in range(10):
            for idx in range(4):
                # golden section search
                a, b = ab_start[idx]
                # calc losses
                coef[idx] = a
                la = self._loss(coef, X, y, idx)
                coef[idx] = b
                lb = self._loss(coef, X, y, idx)
                for it in range(20):
                    # choose value
                    if la > lb:
                        a = b - (b - a) * golden1
                        coef[idx] = a
                        la = self._loss(coef, X, y, idx)
                    else:
                        b = b - (b - a) * golden2
                        coef[idx] = b
                        lb = self._loss(coef, X, y, idx)
        self.coef_ = {'x': coef}

    def predict(self, X, coef):
        X_p = np.array([to_bins(pred, coef) for pred in X])
        return X_p

    def coefficients(self):
        return self.coef_['x']
    
class StratifiedGroupKFold():
    def __init__(self, n_splits=5):
        self.n_splits = n_splits
    
    def split(self, X, y=None, groups=None):
        fold = pd.DataFrame([X, y, groups]).T
        fold.columns = ['X', 'y', 'groups']
        fold['y'] = fold['y'].astype(int)
        g = fold.groupby('groups')['y'].agg('mean').reset_index()
        fold = fold.merge(g, how='left', on='groups', suffixes=('', '_mean'))
        fold['y_mean'] = fold['y_mean'].apply(np.round)
        fold['fold_id'] = 0
        for unique_y in fold['y_mean'].unique():
            mask = fold.y_mean==unique_y
            selected = fold[mask].reset_index(drop=True)
            cv = GroupKFold(n_splits=n_splits)
            for i, (train_index, valid_index) in enumerate(cv.split(range(len(selected)), y=None, groups=selected['groups'])):
                selected.loc[valid_index, 'fold_id'] = i
            fold.loc[mask, 'fold_id'] = selected['fold_id'].values
            
        for i in range(self.n_splits):
            indices = np.arange(len(fold))
            train_index = indices[fold['fold_id'] != i]
            valid_index = indices[fold['fold_id'] == i]
            yield train_index, valid_index
            
def merge(train, test, path, add_cols):
    df_ = feather.read_dataframe(path)
    add_cols += list(df_.columns)
    train = pd.concat((train, df_[:len_train]), axis=1)
    test = pd.concat((test, df_[len_train:].reset_index(drop=True)), axis=1)
    return train, test, add_cols

In [5]:
target = 'AdoptionSpeed'
len_train = 14993
len_test = 3948
    
    
# ===============
# Params
# ===============
seed = 777
kaeru_seed = 1337
n_splits = 5
np.random.seed(seed)

# feature engineering
n_components = 5
img_size = 256
batch_size = 256

Extra_Tree_PARAMS = {
    "max_depth": 5,
    "n_jobs": 16,
    "random_state": seed
}

# define
maxvalue_dict = {}
categorical_features = [
     'Breed1',
     'Breed2',
     'Color1',
     'Color2',
     'Color3',
     'Dewormed',
     'FurLength',
     'Gender',
     'Health',
     'MaturitySize',
     'State',
     'Sterilized',
     'Type',
     'Vaccinated',
     'Type_main_breed',
     'BreedName_main_breed',
     'Type_second_breed',
     'BreedName_second_breed',
]
numerical_features = []
text_features = ['Name', 'Description']
remove = ['index', 'seq_text', 'PetID', 'Name', 'Description', 'RescuerID', 'StateName', 'annots_top_desc', 'Description_Emb', 'Description_bow']

In [11]:
train = feather.read_dataframe('from_kernel/all_datav17.feather')
test = train[len_train:]
train = train[:len_train]

from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder

use_cols = pd.read_csv("importance6.csv")
use_cols["gain"] = use_cols["gain"] / use_cols["gain"].sum()
predictors = list(use_cols[use_cols.gain>0.0002].feature) + add_cols
print(len(predictors))
categorical_features = [c for c in categorical_features if c in predictors]
numerical_features = list(set(predictors) - set(categorical_features + [target] + remove))

X_train = train.append(test).reset_index(drop=True).loc[:, predictors]
for c in categorical_features:
    X_train[c] = LabelEncoder().fit_transform(X_train[c])
X_train.replace(np.inf, np.nan, inplace=True)
X_train.replace(-np.inf, np.nan, inplace=True)
X_train[numerical_features] = StandardScaler().fit_transform(X_train[numerical_features].rank())
X_train.fillna(-999, inplace=True)

X_test = X_train.iloc[len_train:]
X = X_train.iloc[:len_train]

2269


In [30]:
Extra_Tree_PARAMS = {
    "max_depth": 7,
    "n_estimators": 100,
    "n_jobs": 16,
    "random_state": seed
}

In [31]:
%%time

y =  feather.read_dataframe('../input/X_train.feather')["AdoptionSpeed"].values
rescuer_id = pd.read_csv('../input/petfinder-adoption-prediction/train/train.csv').loc[:, 'RescuerID'].iloc[:len_train]

y_pred = np.empty(len_train,)
y_test = []

#cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=1337)
#for fold_id, (train_index, valid_index) in enumerate(cv.split(range(len(X)), y)):
#cv = GroupKFold(n_splits=n_splits)
#for fold_id, (train_index, valid_index) in enumerate(cv.split(range(len(X)), y=None, groups=rescuer_id)): 
cv = StratifiedGroupKFold(n_splits=n_splits)
for fold_id, (train_index, valid_index) in enumerate(cv.split(range(len(X)), y=y, groups=rescuer_id)): 
    X_train = X.loc[train_index, :]
    X_valid = X.loc[valid_index, :]
    y_train = y[train_index]
    y_valid = y[valid_index]

    y_pred_valid, train_rmse, valid_rmse = run_extra_tree_model(X_train, y_train, X_valid, y_valid)
    y_pred_valid = rankdata(y_pred_valid)/len(y_pred_valid)
    y_pred[valid_index] = y_pred_valid.ravel()
    
    print(train_rmse, valid_rmse)


optR = OptimizedRounder()
optR.fit(y_pred, y)
coefficients = optR.coefficients()
y_pred_opt = optR.predict(y_pred, coefficients)
score = get_score(y, y_pred_opt)
print(score)

0.9679393763030208 1.056781568164748
0.9493294034969463 1.1040482498800663
0.9595882782572798 1.0907407168737013
0.9622609477395304 1.0661513055952374
0.9582960109455375 1.0775195178793657
0.3932400660683706
CPU times: user 18min 51s, sys: 1.19 s, total: 18min 52s
Wall time: 1min 34s


In [ ]:
1.0606989278763572